# Automatic (derivatives) for the people and a song of ice and fire

#### Stian Salte, Rebecca Vangen, Jahad Ahmed Hamidi, Ruhan Reza

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math


### Exercise 1

#### Part 1

We import sys and get float info

In [1]:
import sys
print(sys.float_info)
pos_zero = 0.0
print(2**64)

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)
18446744073709551616


Max is the largest representable number in computers, similarily min is the smallest representable number in computers.
Epsilon tells us the smallest number we can add to one to get something other than one.



Machine precision, epsilon is the smallest number we can add to one
and get something different than one, i.e. 1 + epsilon > 1. For a 64-bits
computer this value is epsilon = 2−52 ' 2.2204 × 10−16